In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import transforms

In [2]:
# hyper parameters
batch_size = 32
num_classes = 10
learning_rate = 0.001
num_epochs = 2

In [3]:
# Use transforms.compose method to reformat images for modeling, we are getting a normalization from a range 0 to 1 to -1 to 1
all_transforms = transforms.Compose(
    [
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(
    root="./data", train=True, transform=all_transforms, download=True
)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(
    root="./data", train=False, transform=all_transforms, download=True
)

# Create loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)


test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=True
)

# check number of train/test data
print(len(train_loader))
print(len(test_loader))

Files already downloaded and verified
Files already downloaded and verified
1563
313


In [4]:
# a small trial test for shape parameters
dataiter = iter(train_loader)
images, labels = dataiter.next()

conv1 = nn.Conv2d(3, 6, 5)
pool = nn.MaxPool2d(2, 2)
conv2 = nn.Conv2d(6, 16, 5)
print(images.shape)
trials = conv1(images)
print(trials.shape)
trials = pool(trials)
print(trials.shape)
trials = conv2(trials)
print(trials.shape)
trials = pool(trials)
print(trials.shape)

torch.Size([32, 3, 32, 32])
torch.Size([32, 6, 28, 28])
torch.Size([32, 6, 14, 14])
torch.Size([32, 16, 10, 10])
torch.Size([32, 16, 5, 5])


In [5]:
# Creating a CNN class
class MyCNN(nn.Module):
    #  Determine what layers and their order in CNN object
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=6, kernel_size=5
        )  # Size of kernels, i. e. of size 5x5
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)

        self.fc1 = nn.Linear(400, 128)  # 400 = 16 *5 *5
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    # Progresses data across layers
    def forward(self, x):
        out = self.relu(self.conv1(x))
        out = self.pool(out)

        out = self.relu(self.conv2(out))
        out = self.pool(out)

        out = out.reshape(out.size(0), -1)

        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        return out

In [6]:
model = MyCNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    model.parameters(), lr=learning_rate, weight_decay=0.005, momentum=0.9
)

In [8]:
# training
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    # Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i + 1) % 500 == 0:
            print(
                f"epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():4f}"
            )

    print(
        "Epoch [{}/{}], Loss: {:.4f}".format(
            epoch + 1, num_epochs, loss.item()
        )
    )

epoch 1/2, step 500/1563, loss = 1.880617
epoch 1/2, step 1000/1563, loss = 1.582625
epoch 1/2, step 1500/1563, loss = 1.947143
Epoch [1/2], Loss: 1.1342
epoch 2/2, step 500/1563, loss = 1.695499
epoch 2/2, step 1000/1563, loss = 1.596449
epoch 2/2, step 1500/1563, loss = 1.710818
Epoch [2/2], Loss: 1.3425


In [10]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(
        "Accuracy of the network on the {} test images: {} %".format(
            len(test_loader), 100 * correct / total
        )
    )

    for images, labels in train_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(
        "Accuracy of the network on the {} train images: {} %".format(
            len(train_loader), 100 * correct / total
        )
    )

Accuracy of the network on the 313 test images: 31.19 %
Accuracy of the network on the 1563 train images: 30.736666666666668 %
